## Import libraries

In [2]:
import requests
import re
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

## Unpickle .pkl file back to dataframe

In [3]:
#Raw data
#data = pd.read_pickle("sources/df_articles.pkl")
#data = pd.read_pickle("sources/df_length_of_articles.pkl")

#Data after preprocessing
data = pd.read_pickle("sources/df_articles_ap.pkl")

data.reset_index(inplace = True, drop = True)
data

,category,monitor_name,published_at,raw_body,title,source.name
0,[Z domova],SK - Home news - Unreliable - Wordpress Sites,2019-08-28T13:19:17.000000+0200,<p>Bratislava 28. augusta 2019 (TASR/HSP/Foto:...,"OĽaNO žiada Ministerstvo financií SR, aby pred...",hlavnespravy.sk
1,[],SK - World news - Reliable - RSS + Newspaper p...,2020-10-27T21:14:37.000000+0100,<div><h2>Mnoh&#233; &#353;t&#225;ty protestuj&...,Frustrácia ľudí narastá. Nemali by sme to vzda...,ta3.com
2,"[Úvodná strana (Titulka), Zo zahraničia]",SK - Economic news - Unreliable - Wordpress Sites,2020-10-02T15:00:28.000000+0200,<p>Londýn 2. októbra 2020 (SITA/AP/AFP/HSP/Fot...,Von der Leyenová a Johnson zhodnotia doterajši...,hlavnespravy.sk
3,"[Úvodná strana (Titulka), Zo zahraničia]",SK - World news - Unreliable - Wordpress Sites,2018-06-03T08:19:33.000000+0200,"<p><span style=""color: #333333;"">Novosibirsk 3...",V Rusku predviedli najnovšie obrnené vozidlo T...,hlavnespravy.sk
4,"[Ekonomika, Z domova]",SK - Home news - Unreliable - Wordpress Sites,2018-12-05T12:04:51.000000+0100,<p>Bratislava 5. decembra 2018 (SITA/HSP/Foto:...,Návrhy ministerstva financií prešli parlamentom,hlavnespravy.sk
...,...,...,...,...,...,...
15208,"[Úvodná strana (Titulka), Z domova]",SK - Home news - Unreliable - Wordpress Sites,2020-06-03T15:29:02.000000+0200,<p>Bratislava 3. júna 2020 (TASR/HSP/Foto:Scre...,Program stability si podľa Smeru-SD v mnohom p...,hlavnespravy.sk
15209,[hlavnespravy.sk],SK - Home news - Unreliable - Wordpress Sites,2019-05-20T20:11:11.000000+0200,<p>Bratislava 20. mája 2019 (HSP/Foto:Screensh...,Veci Verejné Romana Michelka: “Slušné” spôsoby...,panobcan.sk
15210,[],SK - Home news - Reliable - RSS + Newspaper pa...,2020-11-24T15:55:36.000000+0100,"<div>\n\n <p class=""perex"">Sul&...",Sulík si nesplnil domácu úlohu. Pripravuje sa ...,sme.sk
15211,[Ekonomika],SK - Economic news - Unreliable - Wordpress Sites,2021-05-31T06:08:20.000000+0200,<p>Bratislava 31. mája 2021 (SITA/HSP/Foto:Pix...,"Využívanie internetu rastie, tento trend bude ...",hlavnespravy.sk


## Load konspiratori.sk list


In [4]:
with open('sources/zoznam_konspiratori.txt') as f:
    lines = f.readlines()
   #print(lines)
    
list_konspiratori = [x.strip() for x in lines] 

## Load mediabiasfactcheck.com list

In [5]:
with open('sources/zoznam_mediabiasfactcheck.txt') as f:
    lines = f.readlines()
   #print(lines)
    
list_mediabiasfactcheck = [x.strip() for x in lines] 

## Merge of the lists

In [6]:
list_konspiratori.extend(list_mediabiasfactcheck)
list_konspiratori

['vigilantcitizen.com',
 'protiprudu.org',
 'biblik.sk',
 'orgo-net.blogspot.sk',
 'aeronet.cz',
 'conspi.cz',
 'nwoo.org',
 'arfa.cz',
 'exopolitika.cz',
 'badatel.net',
 'czechfreepress.cz',
 'qanon.sk',
 'zemavek.sk',
 'stratenaeuropa.sk',
 'bnw-nwo.wz.cz',
 'protiprud.sk',
 'protiproud.cz',
 'blokprotiislamizaci.cz',
 'afinabul.blog.cz',
 'prvopodstata.com',
 'stalo-se.cz',
 'slobodavockovani.sk',
 'magnificat.sk',
 'nekorektni-tv.cz',
 'veksvetla.cz',
 'sho.sk',
 'tadesco.org',
 'nadhlad.com',
 'pravdive.eu',
 'bezpolitickekorektnosti.cz',
 'freeglobe.cz',
 'stop5g.cz',
 'svobodnenoviny.eu',
 'vedomec.com',
 'anti-illuminati.wbl.sk',
 'leva-net.webnode.cz',
 'vlasteneckenoviny.cz',
 'novarepublika.cz',
 'rukojmi.cz',
 'rozhladna.org',
 'yournewswire.com',
 'slobodnyvysielac.sk',
 'ezopress.sk',
 'prekladyodlesa.sk',
 'pokec24.cz',
 'obcianskytribunal.sk',
 'infowars.cz',
 'idemvolit.sk',
 'tadesco.cz',
 'eurasia24.cz',
 'nemesis.sk',
 'euronoviny.eu',
 'zahadnysvet.sk',
 'dobajedu

## Add column with the value of the credibility from konspiratori.sk list


In [45]:
# NON CREDIBLE = 0
# CREDIBLE = 1
new_column = []
for source_name in data['source.name']:
    #print("hladany web je " + source_name)
    for website in list_konspiratori:
        #print("porovnavam weby (" + source_name + "," + website + ")")
        if website == source_name:
            score = 0
            #print ("NASIEL SOM")
            break
        else:
            score = 1
    new_column.append(score)
#print(new_column)
data['source_name_cred'] = new_column
print(data[['source.name', 'source_name_cred']])

           source.name  source_name_cred
0      hlavnespravy.sk                 1
1              ta3.com                 1
2      hlavnespravy.sk                 1
3      hlavnespravy.sk                 1
4      hlavnespravy.sk                 1
...                ...               ...
15208  hlavnespravy.sk                 1
15209      panobcan.sk                 0
15210           sme.sk                 1
15211  hlavnespravy.sk                 1
15212      hnonline.sk                 1

[15213 rows x 2 columns]


## For test

In [ ]:
for i in range(len(data)):
    for website in list_konspiratori:
         domain = re.findall("https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+",data['raw_body'][i])
         print(domain)
        

In [15]:
txt = data['raw_body'][3]
regex = "https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+"
p = re.compile(regex)

domains = re.findall(p,txt)
domains = list(set(domains))

print(txt)
print(domains)
# for domain in domains:
#     print(domain)

text = 'hlavnespravy.sk'

if text in domains[1]:
    print("Found!")
else:
    print("Not found!")


<p><span style="color: #333333;">Novosibirsk 3. júna 2018 (HSP/<a style="color: #333333;" href="https://cz.sputniknews.com/video/201806027467580-vozidlo-tajfun-armada-vojak-video/" target="_blank" rel="noopener noreferrer">Sputnik</a>/Foto:Sputnik)</span></p>
<p>&nbsp;</p>
<p><strong>Najnovšie obrnené vozidlá Tajfun-K sa zúčastnilo cvičenia pri Novosibirsku.</strong></p>
<figure id="attachment_1421714" aria-labelledby="figcaption_attachment_1421714" class="article-image wp-caption"><img class="img-responsive size-full wp-image-1421714" src="https://www.hlavnespravy.sk/wp-content/uploads/2018/06/tajfun-k.jpg" alt="" width="640" height="334" srcset="https://www.hlavnespravy.sk/wp-content/uploads/2018/06/tajfun-k.jpg 640w, https://www.hlavnespravy.sk/wp-content/uploads/2018/06/tajfun-k-300x157.jpg 300w" sizes="(max-width: 640px) 100vw, 640px" /><figcaption id="figcaption_attachment_1421714" class="wp-caption-text image-caption">Na snímke ruské obrnené vozidlo pechoty Tajfun-K.</figcaption

## Check usage of non credible sources in article + add new collumn

In [49]:
new_col_source = []

for article_num in range(len(data)):
# for article_num in range(len(data)):
    domains = []
    regex = "https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+"
    #regex2 = "(zdroj|ZDROJ|SOURCE|source):(?:[-\w.]|(?:%[\da-fA-F]{2}))+"
    p = re.compile(regex)
    #p2 = re.compile(regex2)
    
    raw_body = data['raw_body'][article_num]
    print(raw_body)
    
    domains = re.findall(p,raw_body)
    domains = list(set(domains))
    #domains2 = re.findall(p2,raw_body)
    #domains2 = list(set(domains2))
    
    #domains.extend(domains2)
    
    print(domains)
    for website in list_konspiratori:
        if score == 1:
            break
        for domain in domains:
            if website in domain:
                score = 1
                break
            else:
                score = 0
    new_col_source.append(score)
    score = 0
#print(new_col_source)
data['non_credible_source'] = new_col_source
print(data[['source.name', 'source_name_cred', 'non_credible_source']])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



<p>Bratislava 28. júna 2020 (SITA/HSP/Foto:SITA-Branislav Bibel)</p>
<p>&nbsp;</p>
<p><strong>Andrej Kiska (Za ľudí) nebude na najbližšom sneme strany kandidovať za predsedu strany a končí v politike. Bývalý prezident to dnes oznámil v statuse na sociálnej sieti. Dôvodom ukončenia politickej kariéry je jeho zdravotný stav. Kisku čaká v pondelok 29. júna operácia platničky. Ide pritom o jeho druhú operáciu za posledný rok a tretiu za posledných päť rokov</strong></p>
<figure id="attachment_2162858" aria-labelledby="figcaption_attachment_2162858" class="article-image wp-caption"><img class="img-responsive size-full wp-image-2162858" src="https://admin.hlavnespravy.sk/wp-content/uploads/2020/05/10331.jpg" alt="" width="640" height="469" srcset="https://www.hlavnespravy.sk/wp-content/uploads/2020/05/10331.jpg 640w, https://www.hlavnespravy.sk/wp-content/uploads/2020/05/10331-300x220.jpg 300w" sizes="(max-width: 640px) 100vw, 640px" /><figcaption id="figcaption_attachment_2162858" class="wp

In [47]:
print(data[['source.name', 'source_name_cred', 'non_credible_source']])

           source.name  source_name_cred  non_credible_source
0      hlavnespravy.sk                 1                    1
1              ta3.com                 1                    0
2      hlavnespravy.sk                 1                    0
3      hlavnespravy.sk                 1                    1
4      hlavnespravy.sk                 1                    0
...                ...               ...                  ...
15208  hlavnespravy.sk                 1                    0
15209      panobcan.sk                 0                    1
15210           sme.sk                 1                    0
15211  hlavnespravy.sk                 1                    0
15212      hnonline.sk                 1                    0

[15213 rows x 3 columns]


In [52]:
data['non_credible_source'].value_counts()[0]

12488